In [60]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import gridplot,column
from bokeh.models.formatters import NumeralTickFormatter
import pybithumb
import pyupbit
import pandas as pd
import numpy as np
from IPython.display import HTML

In [146]:
# pybithumb.get_tickers()

In [148]:
# 캔들 만들기

df = pybithumb.get_candlestick("BTC")
df = df.reset_index()

# df = pyupbit.get_ohlcv("KRW-BTC", interval="day1", count = 360)
# df["time"] = df.index
# df


df["time"] = pd.to_datetime(df["time"])

df.rename(columns = {'time' : 'date'}, inplace = True)

df["ma3"] = df["close"].rolling(5).mean()
df["ma5"] = df["close"].rolling(10).mean()
df["ma10"] = df["close"].rolling(20).mean()
df["date"] = pd.to_datetime(df["date"])

# 1.VWAP
v = df['volume'].values
tp = (df['low'] + df['close'] + df['high']).div(3).values
df = df.assign(vwap=(tp * v).cumsum() / v.cumsum())
df["diff"] = df["close"] - df["vwap"] # +면 상승세, -면 하락세

# 2.RSI
df["U"] = np.where(df["close"].diff(1) > 0, df["close"].diff(1), 0)  # 전일대비 상승분 가격
df["D"] = np.where(df["close"].diff(1) < 0, df["close"].diff(1) *(-1), 0) # 전일대비 하락분 가격
# ud_df = pd.DataFrame()
# ud_df["U"] = df["U"] 
# ud_df["D"] = df["D"]

period = 14  # 하락분의 14일 평균을 구해줍니다.
df["AU"] = df["U"].rolling( window=period, min_periods=period ).mean() # 14일간 전일대비 평균 상승가격 
df["AD"] = df["D"].rolling( window=period, min_periods=period ).mean() # 14일간 전일대비 평균 하락가격

# RSI = AU / (AU + AD) 의 백분율을 RSI 로 계산해줍니다.
RSI = (df["AU"] / (df["AU"] + df["AD"])) * 100
df["rsi"] = RSI

# 3.MACD
def fnMACD(m_Df, m_NumFast=12, m_NumSlow=26, m_NumSignal=9):
    m_Df['EMAFast'] = m_Df['close'].ewm( span = m_NumFast, min_periods = m_NumFast - 1).mean() # 12일의 지수평균이동
    m_Df['EMASlow'] = m_Df['close'].ewm( span = m_NumSlow, min_periods = m_NumSlow - 1).mean() # 26
    m_Df['MACD'] = m_Df['EMAFast'] - m_Df['EMASlow'] 
    m_Df['MACDSignal'] = m_Df['MACD'].ewm( span = m_NumSignal, min_periods = m_NumSignal-1).mean() # 9
    m_Df['MACDDiff'] = m_Df['MACD'] - m_Df['MACDSignal']
    return m_Df
df = fnMACD(df)

# 4.BolingerBand
def fnBolingerBand(m_DF, n=20, k=2):
    m_DF['ma20'] = m_DF['close'].rolling(n).mean()
    m_DF['upper'] = m_DF['close'].rolling(n).mean() + k* m_DF['close'].rolling(n).std()
    m_DF['lower'] = m_DF['close'].rolling(n).mean() - k* m_DF['close'].rolling(n).std()
    return m_DF
df = fnBolingerBand(df)


df2 = df.iloc[-360:]
df2

,date,open,close,high,low,volume,ma3,ma5,ma10,vwap,...,AD,rsi,EMAFast,EMASlow,MACD,MACDSignal,MACDDiff,ma20,upper,lower
2442,2020-12-02 00:00:00,21260000.0,20813000.0,21399000.0,20308000.0,5378.511316,20392600.0,20192000.0,19633700.0,7.319544e+06,...,246071.428571,57.766336,2.007442e+07,1.903266e+07,1.041755e+06,1.093750e+06,-51995.342312,19633700.0,2.206961e+07,1.719779e+07
2443,2020-12-03 00:00:00,20813000.0,21102000.0,21240000.0,20660000.0,3438.132952,20786000.0,20295900.0,19796000.0,7.321847e+06,...,246071.428571,56.975147,2.023251e+07,1.918595e+07,1.046560e+06,1.084312e+06,-37751.970641,19796000.0,2.216482e+07,1.742718e+07
2444,2020-12-04 00:00:00,21090000.0,20761000.0,21282000.0,20433000.0,3705.626746,20975600.0,20248000.0,19949000.0,7.324296e+06,...,270428.571429,51.492633,2.031381e+07,1.930262e+07,1.011196e+06,1.069689e+06,-58492.770960,19949000.0,2.213642e+07,1.776158e+07
2445,2020-12-05 00:00:00,20761000.0,20731000.0,20950000.0,20390000.0,2718.414446,20933400.0,20215100.0,20099500.0,7.326074e+06,...,272571.428571,51.089464,2.037800e+07,1.940842e+07,9.695726e+05,1.049666e+06,-80093.095197,20099500.0,2.204195e+07,1.815705e+07
2446,2020-12-06 00:00:00,20731000.0,20785000.0,20977000.0,20585000.0,1736.551209,20838400.0,20408200.0,20238500.0,7.327218e+06,...,238500.000000,54.749966,2.044061e+07,1.951039e+07,9.302200e+05,1.025777e+06,-95556.595560,20238500.0,2.193176e+07,1.854524e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2797,2021-11-22 00:00:00,72991000.0,72416000.0,74179000.0,70764000.0,3875.720534,72569600.0,74613700.0,75729600.0,1.084256e+07,...,842714.285714,29.752903,7.429460e+07,7.437329e+07,-7.868597e+04,8.373404e+05,-916026.330696,75729600.0,8.194996e+07,6.950924e+07
2798,2021-11-23 00:00:00,72416000.0,71500000.0,72592000.0,69394000.0,4399.463100,72238200.0,73896900.0,75657100.0,1.085449e+07,...,908142.857143,22.081265,7.386466e+07,7.416045e+07,-2.957885e+05,6.107146e+05,-906503.091688,75657100.0,8.204540e+07,6.926880e+07
2799,2021-11-24 00:00:00,71500000.0,70275000.0,72000000.0,70050000.0,4210.966338,71896000.0,73106600.0,75466900.0,1.086584e+07,...,995642.857143,14.730532,7.331241e+07,7.387264e+07,-5.602331e+05,3.765251e+05,-936758.126475,75466900.0,8.226632e+07,6.866748e+07
2800,2021-11-25 00:00:00,70280000.0,73152000.0,73514000.0,70230000.0,4565.160279,72067000.0,72585500.0,75413350.0,1.087845e+07,...,800928.571429,32.034186,7.328773e+07,7.381926e+07,-5.315303e+05,1.949140e+05,-726444.275487,75413350.0,8.227064e+07,6.855606e+07


In [149]:
output_notebook()
inc = df2.close >= df2.open # 시세가 증가하는 인덱스
dec = df2.open > df2.close  # 시세가 감소하는 인덱스

candlechart = figure(plot_width = 900, plot_height = 500,
#                      x_axis_type="datetime",
                     x_range = (len(df)-150, len(df)),
#                      y_range=(20000000, 90000000),
                     tools = ['xpan, crosshair, xwheel_zoom, reset, hover, box_select, save'])

candlechart.segment(df2.index[inc], df2.high[inc], df2.index[inc], df2.low[inc], color = "green", line_width = 1)
candlechart.segment(df2.index[dec], df2.high[dec], df2.index[dec], df2.low[dec], color = "green", line_width = 1)
candlechart.vbar(df2.index[inc], 0.5, df2.open[inc], df2.close[inc], fill_color = "blue", line_color = "blue", line_width = 1)
candlechart.vbar(df2.index[dec], 0.5, df2.open[dec], df2.close[dec], fill_color = "red", line_color = "red", line_width = 1)

candlechart.line(df2.index, df2["vwap"], line_color = "orange", legend_label="vwap")
candlechart.line(df2.index, df2["rsi"], line_color = "red", legend_label="RSI")
candlechart.line(df2.index, df2["MACD"], line_color = "blue", legend_label="MACD")

candlechart.line(df2.index, df2["upper"], line_color = "purple", legend_label="Upper Band")
candlechart.line(df2.index, df2["lower"], line_color = "purple", legend_label="Lower Band")
candlechart.legend.location = "top_left"


major_label = {
    i: date.strftime('%Y-%m-%d') for i, date in enumerate(pd.to_datetime(df["date"]))
}

major_label.update({len(df): ""})

candlechart.xaxis.major_label_overrides = major_label

candlechart.yaxis[0].formatter = NumeralTickFormatter(format="0.0")
candlechart.xaxis.axis_label='Date'
candlechart.yaxis.axis_label='Volume'
candlechart.title.text_font_size='25px'
candlechart.title.align='center'

# candlechart.legend


show(candlechart)

Loading BokehJS ...

In [9]:
# output_notebook()
# inc = df.close >= df.open # 시세가 증가하는 인덱스
# dec = df.open > df.close  # 시세가 감소하는 인덱스

# candlechart = figure(plot_width = 900, plot_height = 500, x_range = (a-150, a), tools = ['xpan, crosshair, xwheel_zoom, reset, hover, box_select, save'])

# candlechart.segment(df.index[inc], df.high[inc], df.index[inc], df.low[inc], color = "blue")
# candlechart.segment(df.index[dec], df.high[dec], df.index[dec], df.low[dec], color = "green")
# candlechart.vbar(df.index[inc], 0.5, df.open[inc], df.close[inc], fill_color = "blue", line_color = "blue")
# candlechart.vbar(df.index[dec], 0.5, df.open[dec], df.close[dec], fill_color = "red", line_color = "red")


# major_label = {
#     i: date.strftime('%Y-%m-%d') for i, date in enumerate(pd.to_datetime(df["date"]))
# }

# major_label.update({len(df2): ""})
# candlechart.xaxis.major_label_overrides = major_label
# candlechart.yaxis[0].formatter = NumeralTickFormatter(format="0.0")
# candlechart.xaxis.axis_label='Date'
# candlechart.yaxis.axis_label='Price'
# candlechart.title.text='Price Candle Stick'
# candlechart.title.text_font_size='25px'
# candlechart.title.align='center'

# p = gridplot([[candlechart]])
# show(p)

In [150]:
barchart = figure(plot_width = 900, plot_height = 300, x_range = (len(df)-150, len(df)), 
                  y_range=(0, 15000),
                  tools = ['xpan, crosshair, xwheel_zoom, reset, hover, box_select, save'])
barchart.vbar(df2.index[inc], 0.5, df2[inc]["volume"], fill_color="red", line_color="red")
barchart.vbar(df2.index[dec], 0.5, df2[dec]["volume"], fill_color="blue", line_color="blue")

barchart.xaxis.major_label_overrides = major_label
barchart.yaxis[0].formatter = NumeralTickFormatter(format="0.0")
barchart.xaxis.axis_label='Date'
barchart.yaxis.axis_label='Volume'
barchart.title.text_font_size='25px'
barchart.title.align='center'

show(barchart)


In [55]:
q = figure(plot_width = 1000, plot_height = 400, 
           x_axis_type="datetime",
           tools = ['xpan, crosshair, xwheel_zoom, reset, hover, box_select, save'])

# x_axis_type="datetime"
q.yaxis[0].formatter = NumeralTickFormatter(format="0.0")

q.xaxis.major_label_overrides = major_label
q.xaxis.axis_label='Date'
q.yaxis.axis_label='Price'
q.title.text='Price Trend'
q.title.text_font_size='20px'
q.title.align='center'

q.line(df2["date"], df2["close"], color = 'navy', alpha=0.5, line_width = 2)

show(q)


In [16]:
df2

,date,open,close,high,low,volume,ma3,ma5,ma10,vwap,diff
2441,2020-12-01 00:00:00,20942000.0,21260000.0,21622000.0,20111000.0,8987.171344,19978000.0,20119100.0,19476050.0,7.315982e+06,1.394402e+07
2442,2020-12-02 00:00:00,21260000.0,20813000.0,21399000.0,20308000.0,5378.511316,20392600.0,20192000.0,19633700.0,7.319544e+06,1.349346e+07
2443,2020-12-03 00:00:00,20813000.0,21102000.0,21240000.0,20660000.0,3438.132952,20786000.0,20295900.0,19796000.0,7.321847e+06,1.378015e+07
2444,2020-12-04 00:00:00,21090000.0,20761000.0,21282000.0,20433000.0,3705.626746,20975600.0,20248000.0,19949000.0,7.324296e+06,1.343670e+07
2445,2020-12-05 00:00:00,20761000.0,20731000.0,20950000.0,20390000.0,2718.414446,20933400.0,20215100.0,20099500.0,7.326074e+06,1.340493e+07
...,...,...,...,...,...,...,...,...,...,...,...
2796,2021-11-21 00:00:00,72309000.0,72992000.0,74000000.0,71360000.0,3050.005716,72698800.0,75126700.0,75835950.0,1.083182e+07,6.216018e+07
2797,2021-11-22 00:00:00,72991000.0,72416000.0,74179000.0,70764000.0,3875.720534,72569600.0,74613700.0,75729600.0,1.084256e+07,6.157344e+07
2798,2021-11-23 00:00:00,72416000.0,71500000.0,72592000.0,69394000.0,4399.463100,72238200.0,73896900.0,75657100.0,1.085449e+07,6.064551e+07
2799,2021-11-24 00:00:00,71500000.0,70275000.0,72000000.0,70050000.0,4210.966338,71896000.0,73106600.0,75466900.0,1.086584e+07,5.940916e+07


In [28]:
e = figure(plot_width = 950, plot_height = 400, x_axis_type="datetime" ,tools = ['xpan, crosshair, xwheel_zoom, reset, hover, box_select, save'])
e.yaxis[0].formatter = NumeralTickFormatter(format="0.0")
e.xaxis.axis_label='Date'
e.yaxis.axis_label='Price'
e.title.text='Moving Average'
e.title.text_font_size='20px'
e.title.align='center'

e.line(df2["date"], df2["close"],color = 'blue', alpha=0.5, line_width = 1, legend_label = "close")
e.line(df2["date"], df2["ma3"],color = 'green', alpha=0.5, line_width = 1, legend_label = "MA3")
e.line(df2["date"], df2["ma5"],color = 'red', alpha=0.5, line_width = 1, legend_label = "MA5")
e.line(df2["date"], df2["ma10"],color = 'purple', alpha=0.5, line_width = 1, legend_label = "MA10")
e.legend.location = "top_left"

show(e)


In [56]:
kk = gridplot([[candlechart],[barchart]])
show(kk)

In [5]:
import jinja2
from bokeh.embed import components

template = jinja2.Template('''\
<html>
<link href = "http://cdn.pydata.org/bokeh/release/bokeh-1.4.0.min.css" rel = "stylesheet" type = "text/css">
<script src = "http://cdn.pydata.org/bokeh/release/bokeh-1.4.0.min.js"></script>

<head>
<title> bokeh test </title>
</head>

<body>
<h1> This site for graph test </h1>
파이썬 웹 보케 테스트
{{script}}
{{div}}

</body>
</html>
'''
)

In [15]:
p

Column(id='1076', ...)

In [8]:
script, div = components(p)

HTML(template.render(script=script, div=div))

In [7]:
with open('a.html', 'w') as f:
    f.write(template.render(script=script, div=div))

import webbrowser

webbrowser.open('a.html')

True

In [31]:
# 여러 가상화폐 정보
all = pybithumb.get_current_price("ALL") 
for ticker, data in all.items():
    print(ticker, data['closing_price'])

BTC 51985000
ETH 3568000
LTC 185700
ETC 66650
XRP 944.5
BCH 669500
QTUM 12010
BTG 66800
EOS 5155
ICX 1251
TRX 84.26
ELF 306.1
OMG 5760
KNC 1963
GLM 480
ZIL 108.6
WAXP 204.2
POWR 347.2
LRC 325.6
STEEM 602.3
STRAX 2308
ZRX 1086
REP 30840
XEM 207.2
SNT 100.4
ADA 1779
CTXC 228.4
BAT 808.6
WTC 760
THETA 7775
LOOM 101.5
WAVES 19750
TRUE 173.9
LINK 27840
ENJ 1791
VET 119.7
MTL 2666
IOST 33.4
TMTG 2.772
QKC 22.17
HDAC 40.4
AMO 3.085
BSV 183300
DAC 3.161
ORBS 102.5
TFUEL 342.9
VALOR 469.1
CON 3.278
ANKR 108.8
MIX 4.923
CRO 162.6
FX 427.9
CHR 376.4
MBL 10.41
MXC 39.42
FCT 138.8
TRV 17.51
DAD 224.9
WOM 121.4
SOC 30.21
EM 6.096
BOA 141.2
FLETA 14.85
SXP 3381
COS 18.3
APIX 57.71
EL 8.969
BASIC 5.37
HIVE 510.4
XPR 4.369
VRA 20.17
FIT 1.804
EGG 4.577
BORA 260.5
ARPA 53.12
APM 36.8
CKB 14.98
AERGO 224.1
ANW 54.18
CENNZ 94.8
EVZ 53.16
CYCLUB 34.65
SRM 5280
QTCON 18.25
UNI 32990
YFI 39407000
UMA 12450
AAVE 434800
COMP 536500
REN 522.5
BAL 28100
RSR 43.92
NMR 45120
RLC 4176
SAND 756.6
CVT 37.3
GOM2 10.93